In [1]:
# Training notebook for 1Dconv 16x16 Full model with filtered Dataset3sr and input dig.
# Note that it is currently configured to work with 20 timeslices

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-07-21 00:19:06.374569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-21 00:19:06.374623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-21 00:19:06.375646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-21 00:19:06.382361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-21 00:19:09.858227: W tensorflow/compiler/tf2

In [2]:
from full_1D.Full_nll4 import *
#from models.models import *
from full_1D.Full1D_model2 import *
import utils
from OptimizedDataGenerator_v2 import OptimizedDataGenerator

2025-07-21 00:19:27.612995: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1186 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 1g.5gb, pci bus id: 0000:81:00.0, compute capability: 8.0


In [4]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 68
val_file_size = 12

Training

In [5]:
def count_files_in_directory(directory_path):
    try:
        return len([f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))])
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return 0


In [6]:
dataset_base_dir = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/"

data_directory_path = os.path.join(dataset_base_dir, "train_filtered")
data_directory_path_2 = os.path.join(dataset_base_dir, "test_filtered")

tfrecords_dir_train = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/train_filtered/"
tfrecords_dir_validation = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/test_filtered/"

In [7]:
# Count files in both directories
data_files_count = count_files_in_directory(data_directory_path)

# Print results
print(f"Number of files in data directory: {data_files_count}")

Number of files in data directory: 80


Create a new tfrecord can avoid the problem of not converging. 

In [8]:
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = data_directory_path_2,
    is_directory_recursive = False,
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize = True,
    include_y_local = False,
    # Labels you want to extract
    labels_list = ['x-midplane', 'y-midplane', 'cotAlpha', 'cotBeta'],
    # Input shape for the model (time slices, height, width)
    input_shape = (20, 16, 16),
    transpose = (0, 2, 3, 1),
    shuffle = False,
    files_from_end = False,
    
    # New parameters specific to this generator
    tfrecords_dir = tfrecords_dir_validation,
    tfrecords_dir_save = "/home/callea/TFrecords_3sr_qf_full_20t/val", 
    use_time_stamps = list(range(20)),
    #[0, 19],
    # Specific time stamps to use
    quantize = True,  # Whether to use quantization during loading
    max_workers = 2  # For parallel processing
)

Saving batches as TFRecords: 100%|██████████| 14/14 [00:21<00:00,  1.56s/it]


Metadata saved successfully ast /home/callea/TFrecords_3sr_qf_full_20t/val/metadata.json
Loading metadata from /home/callea/TFrecords_3sr_qf_full_20t/val/metadata.json


In [9]:
training_generator = OptimizedDataGenerator(
    # Base directory where your parquet files are located
    dataset_base_dir = data_directory_path,
    is_directory_recursive = False,
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize = True,
    include_y_local = False,
    # Labels you want to extract
    labels_list = ['x-midplane', 'y-midplane', 'cotAlpha', 'cotBeta'],
    # Input shape for the model (time slices, height, width)
    input_shape = (20, 16, 16),
    transpose = (0, 2, 3, 1),
    shuffle = False,
    files_from_end = True,  # Using files from the end of the list
    
    # New parameters specific to this generator
    tfrecords_dir = tfrecords_dir_train,
    tfrecords_dir_save = "/home/callea/TFrecords_3sr_qf_full_20t/train", 
    use_time_stamps = list(range(20)),
    #[0, 19],  # Specific time stamps to use
    quantize = True,  # Whether to use quantization during loading
    max_workers = 2  # For parallel processing
)

Saving batches as TFRecords: 100%|██████████| 74/74 [02:06<00:00,  1.71s/it]


Metadata saved successfully ast /home/callea/TFrecords_3sr_qf_full_20t/train/metadata.json
Loading metadata from /home/callea/TFrecords_3sr_qf_full_20t/train/metadata.json


In [10]:
input_shape = (16, 16, 20)
model = CreateModel_full(input_shape)  # Use Full model
model.summary()

Model: "smrtpxl_regression_full"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_pxls/ (InputLayer)    [(None, 16, 16, 20)]         0         []                            
                                                                                                  
 avg_pooling_2d_proj_x (Ave  (None, 16, 1, 20)            0         ['input_pxls/[0][0]']         
 ragePooling2D)                                                                                   
                                                                                                  
 avg_pooling_2d_proj_y (Ave  (None, 1, 16, 20)            0         ['input_pxls/[0][0]']         
 ragePooling2D)                                                                                   
                                                                            

In [11]:
# Try a much higher learning rate
model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),  # 10x higher!
    loss=custom_diag_loss
)

In [12]:
fingerprint = '%08x' % random.randrange(16**8)
os.makedirs("trained_models", exist_ok=True)
base_dir = f'./trained_models/model-{fingerprint}-checkpoints'
os.makedirs(base_dir, exist_ok=True)  
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [13]:
checkpoint_filepath

'./trained_models/model-1ba7272a-checkpoints/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [14]:
print(fingerprint)

1ba7272a


In [15]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, Callback

early_stopping_patience = 50

class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        checkpoints = [f for f in os.listdir(base_dir) if f.startswith('weights')]
        if len(checkpoints) > 1:
            checkpoints.sort()
            for checkpoint in checkpoints[:-1]:
                os.remove(os.path.join(base_dir, checkpoint))

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

csv_logger = CSVLogger(f'{base_dir}/training_log.csv', append=True)

In [16]:
class NanMonitorCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print('NaN loss detected, terminating training')
                self.model.stop_training = True
                
# Add this to your callbacks list
nan_monitor = NanMonitorCallback()
callbacks = [mcp, csv_logger, nan_monitor]

In [ ]:
# Now use it in model.fit()
history = model.fit(
    x=training_generator,
    validation_data=validation_generator,
    callbacks=callbacks,
    epochs=1000,
    shuffle=False,
    verbose=1
)

2025-07-21 00:23:29.391412: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/1000


2025-07-21 00:23:32.159125: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-07-21 00:23:32.273042: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-07-21 00:23:33.190645: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f89831ac730 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-21 00:23:33.190709: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB MIG 1g.5gb, Compute Capability 8.0
2025-07-21 00:23:33.202687: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753050213.324132  850563 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 7172.5288
Epoch 1: val_loss improved from inf to -4289.93506, saving model to ./trained_models/model-1ba7272a-checkpoints/weights.01-t7172.53-v-4289.94.hdf5
74/74 [==============================] - 29s 341ms/step - loss: 7172.5288 - val_loss: -4289.9351
Epoch 2/1000
74/74 [==============================] - ETA: 0s - loss: -10463.6299
Epoch 2: val_loss improved from -4289.93506 to -14859.66211, saving model to ./trained_models/model-1ba7272a-checkpoints/weights.02-t-10463.63-v-14859.66.hdf5
74/74 [==============================] - 20s 273ms/step - loss: -10463.6299 - val_loss: -14859.6621
Epoch 3/1000
74/74 [==============================] - ETA: 0s - loss: -18130.7930
Epoch 3: val_loss improved from -14859.66211 to -19149.19922, saving model to ./trained_models/model-1ba7272a-checkpoints/weights.03-t-18130.79-v-19149.20.hdf5
74/74 [==============================] - 24s 326ms/step - loss: -18130.7930 - val_loss: -19149.1992
Epoch 